# UCI MTB DH Data Retrieval

## Setup
#### Import Libraries

If you do not have these libraries available, you should install them using `pip`

```
pip install requests
pip install bs4
pip install pandas
```

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import os

In [5]:
def calculate_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

Widen display area to prevent column wrapping, and always show all columns for debug 

In [6]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

## Config

Which race data are we collecting?

1. Losinj
1. Fort William
1. Leogang
1. Val di Sole
1. Vallnord
1. Mont-Sainte-Anne
1. La Bresse

In [79]:
year = 2018
race = 3
gender = 'f'
event = 'dh'
rnrSpeed = True

#### Data Sources

The UCI Live Timing API contains a lot of data points, but not all the ones we want (speed being the main one missing), and not even all the ones they include on their own PDF which is frustrating.

Similarly, Roots & Rain also has a lot of the data points, but again not all of them; most notably it's missing timing splits 4 and 5.

Therefore we need to pull from both sources and combine the sets.

We specify the URLs for both sources from which we will extract our data. The UCI API URL can be found by loading the Live Timing page then using your browser's inspector on the Network tab (in Chrome at least) to see the data feed. As the UCI seems to be using a Single Page Application (SPA) here, it's not straight forward to extract this link automagically.

**Note:** The Race list is now maintained as an external python config file `config.py` imported below

In [80]:
from config import races

racename = races[year][race]['name']
urlUci = races[year][race]['urls']['uci'] + str(( 3 if 'm' == gender else 6 )) + '/'
urlUciQ = races[year][race]['urls']['uci'] + str(( 2 if 'm' == gender else 5 )) + '/'
urlRoots = races[year][race]['urls']['rnr'] + gender + '/'

File handling setup

In [81]:
directory = event + str(race) + '_' + racename
if not os.path.exists(directory):
    os.makedirs(directory)

file_prefix = str(year) + '_' + event + str(race) + '_' + racename + '_' + gender
file_prefix = os.path.join( directory, file_prefix )

# UCI API
### Load Data

These two lines make the actual request to the server, and then converts the JSON string response in to a usable list format (deserialization)

In [82]:
r = requests.get( urlUci ).json()
q = requests.get( urlUciQ ).json()

The API returns with three main sections:

1. `Last Finisher`
 - Racers in order of start time
2. `Results`
 - Racers in finishing rank order
3. `Riders`
 - Personal details on all racers
 
Each contains many data points. To see all the contained data, you can un-comment and execute any of the lines in the next section to explore more.

In [83]:
# display( q )
# display( d['Results'][7] )
# display( d['Riders']['1001'] )
# display( d['Results'][61] )

### Extract Data

Here we iterate over the `Results` sub-set of data to extract the information we care about: basically some identifying info, and their splits.

There is a loop within a loop here as the first iterates over the two result sets qualifying and race, and within that we extract the necessary stats.

If you looked at detail of the returned data set in the last step you might have noticed the rider's name is not stored next to their result, riders are only identified by a reference number. To facilitate our analysis later on it is useful to import each rider's name at this stage by cross-referencing the `Riders` sub-set.

We start with an empty list `lst` and in each loop iteration add an entry (actually a dict) to that list for each rider.

In [84]:
dat = {}
for i, d in enumerate( [ r, q ] ):
    lastStart = d['Riders'][list(d['Riders'].keys())[-1]]['StartOrder']
    pfx = 'q_' if 1 == i else ''

    splits = len(d['Results'][0]['Times'] )
    lst = []
    for idx, row in enumerate( d['Results'] ):
        fin = "Finished" == row['Status']
        res = {
            'name': d['Riders'][str(row['RaceNr'])]['PrintName'],
            'id': row['RaceNr'],
            'uci': d['Riders'][str(row['RaceNr'])]['UciRiderId'],
            'bib': d['Riders'][str(row['RaceNr'])]['RaceNr'],
            pfx + 'status': row['Status'],
            pfx + 'rank': row['Position'] if fin else idx+1,
            pfx + 'start': d['Riders'][str(row['RaceNr'])]['StartOrder'],
            pfx + 'start_rev': lastStart - d['Riders'][str(row['RaceNr'])]['StartOrder'] +1
        }
        if rnrSpeed:
            res[pfx + 'speed'] = np.nan

        # Add all splits to result set
        for split in range( 0, splits ):
            head = pfx + 'split'
            res[head + str(split+1)] = row['Times'][split]['RaceTime']/1000 if fin else np.nan

        # Append result set to list
        lst.append(res)

    dat[i] = lst

Here we load the completed lists in to Pandas dataframes to facilitate working with the data moving forward

In [85]:
df = pd.DataFrame( dat[0] )
dq = pd.DataFrame( dat[1] )

# Points

Neither data set contains points awarded so we use a reference file and merge

Merge type here must be `outer` so people that finished outside the top 60 men, top 15 women, or DNF, don't get trimmed from the dataset

In [86]:
df_qp = pd.read_csv( event + '_points_qual_' + gender + '.csv', index_col=0 )
df_qp = df_qp.reset_index(drop=False)
dq = dq.merge( df_qp, left_index=True, right_index=True, how="outer")

df_rp = pd.read_csv( event + '_points_race_' + gender + '.csv', index_col=0 )
df_rp = df_rp.reset_index(drop=False)
df = df.merge( df_rp, left_index=True, right_index=True, how="outer")

In [87]:
display( df.head(), dq.head() )

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,r_points
0,2,2002,ATHERTON Rachel,1,NaN,51.093,92.842,135.011,186.012,221.141,10,8,Finished,10003434487,200
1,1,2001,NICOLE Myriam,2,NaN,51.993,93.250,135.512,185.509,221.779,13,5,Finished,10004535237,160
2,6,2006,HANNAH Tracey,3,NaN,52.512,94.542,137.550,190.704,225.118,16,2,Finished,10003732258,140
3,9,2009,HRASTNIK Monika,4,NaN,55.303,99.647,142.596,194.214,229.597,14,4,Finished,10017602955,125
4,7,2007,SIEGENTHALER Emilie,5,NaN,54.994,98.833,143.450,195.311,232.780,15,3,Finished,10004167243,110


,bib,id,name,q_rank,q_speed,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points
0,3,2003,SEAGRAVE Tahnee,1,NaN,53.771,97.831,139.726,191.741,227.228,3,39,Finished,10007414016,50.0
1,6,2006,HANNAH Tracey,2,NaN,55.436,99.536,143.504,195.101,230.608,5,37,Finished,10003732258,40.0
2,7,2007,SIEGENTHALER Emilie,3,NaN,55.776,99.845,144.556,197.387,235.296,6,36,Finished,10004167243,30.0
3,9,2009,HRASTNIK Monika,4,NaN,56.999,100.942,144.646,198.395,235.390,8,34,Finished,10017602955,25.0
4,1,2001,NICOLE Myriam,5,NaN,52.456,103.344,146.517,202.422,238.608,1,41,Finished,10004535237,20.0


# Merge and Expand

This code merges the qualifying and race data in to a single data frame, merging only the columns that are unique between them to avoid duplicates. This allows us to do more in depth analysis later on.

As we merged race in to quali, we re-sort the resulting dataset by race rank

In [88]:
dfq = dq.merge( df[['id'] + list(df.columns.difference( dq.columns ))], left_on='id', right_on='id', how='outer' )
dfq = dfq.sort_values( 'rank', ascending=True )
dfq = dfq.reset_index( drop=True )
dfq['points'] = dfq['r_points'].fillna(0) + dfq['q_points'].fillna(0)

# Time difference between race and quali
dfq['qr_diff'] = dfq['split5'] - dfq['q_split5']

#### Expand Dataset

Calculate and add all the extra columns we need for split and sector differences and their rankings

In [89]:
for pfx in [ 'q_', '' ]:
    for i in range( 1, splits+1 ):
        split = pfx + 'split' + str(i)
        sector = split + '_sector'
        dfq[split + '_rank'] = dfq[split].rank(method='dense')
        dfq[split + '_vs_best'] = (dfq[split] - dfq[split].min())
        dfq[split + '_vs_winner'] = (dfq[split] - dfq[split][0])

        if i > 1:
            dfq[split + '_sector'] = dfq[split] - dfq[pfx + 'split' + str(i-1)]
            dfq[split + '_sector_rank'] = dfq[sector].rank(method='dense')
            dfq[split + '_sector_vs_best'] = (dfq[sector] - dfq[sector].min())
            dfq[split + '_sector_vs_winner'] = (dfq[sector] - dfq[sector][0])

We can take a peek at our data at this point to make sure it looks how we expect.

At this point the `speed` column is NaN (Not a Number) for all racers. This will be filled in below.

In [90]:
display( dfq.head(10) )

,bib,id,name,q_rank,q_speed,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points,r_points,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,points,qr_diff,q_split1_rank,q_split1_vs_best,q_split1_vs_winner,q_split2_rank,q_split2_vs_best,q_split2_vs_winner,q_split2_sector,q_split2_sector_rank,q_split2_sector_vs_best,q_split2_sector_vs_winner,q_split3_rank,q_split3_vs_best,q_split3_vs_winner,q_split3_sector,q_split3_sector_rank,q_split3_sector_vs_best,q_split3_sector_vs_winner,q_split4_rank,q_split4_vs_best,q_split4_vs_winner,q_split4_sector,q_split4_sector_rank,q_split4_sector_vs_best,q_split4_sector_vs_winner,q_split5_rank,q_split5_vs_best,q_split5_vs_winner,q_split5_sector,q_split5_sector_rank,q_split5_sector_vs_best,q_split5_sector_vs_winner,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner
0,2,2002,ATHERTON Rachel,16,NaN,74.580,116.942,160.266,212.906,249.889,2,40,Finished,10003434487,NaN,200.0,1.0,NaN,51.093,92.842,135.011,186.012,221.141,10.0,8.0,Finished,200.0,-28.748,37.0,22.124,0.000,31.0,19.111,0.000,42.362,1.0,0.000,0.000,30.0,20.540,0.000,43.324,3.0,1.429,0.000,18.0,21.165,0.000,52.640,3.0,1.043,0.000,16.0,22.661,0.000,36.983,5.0,1.496,0.000,1.0,0.000,0.000,1.0,0.000,0.000,41.749,2.0,0.492,0.000,1.0,0.000,0.000,42.169,1.0,0.000,0.000,2.0,0.503,0.000,51.001,2.0,1.004,0.000,1.0,0.000,0.000,35.129,2.0,0.715,0.000
1,1,2001,NICOLE Myriam,5,NaN,52.456,103.344,146.517,202.422,238.608,1,41,Finished,10004535237,20.0,160.0,2.0,NaN,51.993,93.250,135.512,185.509,221.779,13.0,5.0,Finished,180.0,-16.829,1.0,0.000,-22.124,8.0,5.513,-13.598,50.888,32.0,8.526,8.526,5.0,6.791,-13.749,43.173,2.0,1.278,-0.151,7.0,10.681,-10.484,55.905,14.0,4.308,3.265,5.0,11.380,-11.281,36.186,3.0,0.699,-0.797,2.0,0.900,0.900,2.0,0.408,0.408,41.257,1.0,0.000,-0.492,2.0,0.501,0.501,42.262,2.0,0.093,0.093,1.0,0.000,-0.503,49.997,1.0,0.000,-1.004,2.0,0.638,0.638,36.270,4.0,1.856,1.141
2,6,2006,HANNAH Tracey,2,NaN,55.436,99.536,143.504,195.101,230.608,5,37,Finished,10003732258,40.0,140.0,3.0,NaN,52.512,94.542,137.550,190.704,225.118,16.0,2.0,Finished,180.0,-5.490,3.0,2.980,-19.144,2.0,1.705,-17.406,44.100,5.0,1.738,1.738,2.0,3.778,-16.762,43.968,5.0,2.073,0.644,2.0,3.360,-17.805,51.597,1.0,0.000,-1.043,2.0,3.380,-19.281,35.507,2.0,0.020,-1.476,3.0,1.419,1.419,3.0,1.700,1.700,42.030,3.0,0.773,0.281,3.0,2.539,2.539,43.008,4.0,0.839,0.839,3.0,5.195,4.692,53.154,6.0,3.157,2.153,3.0,3.977,3.977,34.414,1.0,0.000,-0.715
3,9,2009,HRASTNIK Monika,4,NaN,56.999,100.942,144.646,198.395,235.390,8,34,Finished,10017602955,25.0,125.0,4.0,NaN,55.303,99.647,142.596,194.214,229.597,14.0,4.0,Finished,150.0,-5.793,5.0,4.543,-17.581,4.0,3.111,-16.000,43.943,2.0,1.581,1.581,4.0,4.920,-15.620,43.704,4.0,1.809,0.380,4.0,6.654,-14.511,53.749,5.0,2.152,1.109,4.0,8.162,-14.499,36.995,6.0,1.508,0.012,6.0,4.210,4.210,5.0,6.805,6.805,44.344,7.0,3.087,2.595,4.0,7.585,7.585,42.949,3.0,0.780,0.780,4.0,8.705,8.202,51.618,3.0,1.621,0.617,4.0,8.456,8.456,35.383,3.0,0.969,0.254
4,7,2007,SIEGENTHALER Emilie,3,NaN,55.776,99.845,144.556,197.387,235.296,6,36,Finished,10004167243,30.0,110.0,5.0,NaN,54.994,98.833,143.450,195.311,232.780,15.0,3.0,Finished,140.0,-2.516,4.0,3.320,-18.804,3.0,2.014,-17.097,44.069,4.0,1.707,1.707,3.0,4.830,-15.710,44.711,9.0,2.816,1.387,3.0,5.646,-15.519,52.831,4.0,1.234,0.191,3.0,8.068,-14.593,37.909,10.0,2.422,0.926,5.0,3.901,3.901,4.0,5.991,5.991,43.839,4.0,2.582,2.090,5.0,8.439,8.439,44.617,9.0,2.448,2.448,5.0,9.802,9.299,51.861,4.0,1.864,

#### Rider Data

Saving the personal information about each racer is much easier as we can just export the entire `Riders` dataset. However, the rows and columns are the wrong way round so the `.T` command *transposes* the information, meaning it basically flips the axes.

In [91]:
df2 = pd.DataFrame( d['Riders'] )
df2 = df2.T
df2['Age'] = [ calculate_age( dt.datetime.strptime( dob[:10], "%Y-%m-%d" ) ) for dob in df2['BirthDate'] ]

Here we can glimpse the first few rows of our `DataFrame` and can check the data looks as we expect

In [92]:
display( df2.head() )

,BirthDate,CategoryCode,FamilyName,GivenName,Id,Nation,Outfit,PrintName,Protected,RaceId,RaceNr,ScoreboardName,StartOrder,StartTime,UciCode,UciRank,UciRiderId,UciTeamCode,UciTeamId,UciTeamName,WorldCupRank,Age
2001,1990-02-08T00:00:00,WE,NICOLE,Myriam,1197024565271776,FRA,WCL,NICOLE Myriam,False,0,1,NICOLE M,1,48600000,FRA19900208,1,10004535237,CVN,1590,COMMENCAL / VALLNORD,1,28
2002,1987-12-06T00:00:00,WE,ATHERTON,Rachel,1197024565271777,GBR,NCh,ATHERTON Rachel,False,0,2,ATHERTON R,2,48630000,GBR19871206,5,10003434487,TDH,1598,TREK FACTORY RACING DH,2,30
2003,1995-06-15T00:00:00,WE,SEAGRAVE,Tahnee,1197024565271778,GBR,NaN,SEAGRAVE Tahnee,False,0,3,SEAGRAVE T,3,48660000,GBR19950615,2,10007414016,FMD,1863,TRANSITION BIKES / MUC-OFF FACTORY RACING,3,23
2004,1997-03-12T00:00:00,WE,CABIROU,Marine,1197024565271779,FRA,NaN,CABIROU Marine,False,0,4,CABIROU M,4,48690000,FRA19970312,4,10009563069,VVR,2096,VOULVOUL RACING,4,21
2006,1988-06-13T00:00:00,WE,HANNAH,Tracey,1197024565271780,AUS,NCh,HANNAH Tracey,False,0,6,HANNAH T,5,48720000,AUS19880613,3,10003732258,URT,1608,POLYGON UR,6,30


# Speed Data

Roots and Rain seem to take about 3 days to get their results online. Given all UCI data is available immediately I have added a second method for getting speed data. There is boolean in the config at top of this notebook to decide if we pull data from RnR or we use an import CSV file.

## Roots and Rain

### Load Data

Similar to the UCI api, we make a request to the server with the previously declared `urlRoots` variable. This time however we simply load the content of the response as text which is actually the HTML code of the web page. We do not do have a nice JSON API to read which means we will not deserialize.

Next we invoke a utility called `BeautifulSoup` to help us extract the data from this messy HTML code

In [93]:
if rnrSpeed:
    c = requests.post( urlRoots ).content
    soup = BeautifulSoup( c, "html.parser" )

### Extract Data

If you look at the Roots and Rain page you'll see it listed in a tabular format. What we do here is find all the rows of that table so we can extract the information we need.

Specifically we are looking for instances of `tr` (table row), with a class that *begins with* `c-` as this is a common denomenator I discovered when looking through the code with the browser inspector

In [94]:
if rnrSpeed:
    rows = soup.find_all( "tr", class_=lambda x: x and 'c-' in x )

Similar to the UCI data set, here we will iterate over each row in our data set--basically each table row from the web page--and extract the bits we need.

Racer speed is the metric we're interested in, but in order to match that to our existing data set we need a corresponding identifier so we also extract the racer licence number as that exists in both sets and we can match them together: it is the *intersect* between both sets of data.

To summarise:
1. Extract licence number and corresponding speed
2. Import speed to existing DataFrame matching racers by licence

The `if` condition in the middle will exit this block of code once we hit the end of the Elite finishers, seeing as that's all we have in our existing data set so can't match anyone else

In [95]:
if rnrSpeed:
    for row in rows:
        cells = row.find_all( "td" )
        qspd = cells[7].text[:5]
        spd = cells[12].text[:5]
        qspeed = float( qspd if 0 < len(qspd) else 0 )
        speed = float( spd if 0 < len(spd) else 0 )
        licence = cells[4].text
        bib = int( cells[1].text )
        pos = cells[0].text[8:]
        if "" == pos: break

        # Match rider by UCI licence if present, otherwise fallback to bib
        if len(df2.loc[df2['UciRiderId'] == licence].index.values ):
            rid = int(df2.loc[df2['UciRiderId'] == licence].index.values[0])
        else:
            rid = int( df2.loc[df2['RaceNr'] == bib].index.values[0] )

        # Add speed, and other associated metrics
        dfq.loc[dfq['id'] == rid, 'speed'] = speed
        dfq.loc[dfq['id'] == rid, 'q_speed'] = qspeed

As before, we can take another look at how our data is looking, with the `speed` column now containing data 

## UCI PDF Converted Speed

Despite UCI having a speed field in the splits data of their API, it is always 0. Thanks. They do make that data available in their PDFs, but that data is not easy to extract and all regular converters fail. However, trying with some OCR engines I did have good success. The best of which is https://convertio.co/ocr/. I take the converted file, strip it down to UCI# and speed, save as CSV, and then import and merge here.

Regex code for removing (X) rank from OCR converted files.

> Find: `(,[0-9\.]+).*`
>
> Replace: `$1`

In [96]:
if not rnrSpeed:
    dfs = pd.read_csv( file_prefix + '.speeds.csv' )
    dfsq = pd.read_csv( file_prefix + '.qspeeds.csv' )
    dfq.uci = dfq.uci.astype(str)
    dfs.uci = dfs.uci.astype(str)
    dfsq.uci = dfsq.uci.astype(str)

    dfq = dfq.merge( dfs, left_on='uci', right_on='uci', how='left' )
    dfq = dfq.merge( dfsq, left_on='uci', right_on='uci', how='left' )
    # dfqs[['name', 'uci', 'q_speed', 'speed']]
    # dfqs.columns

In [97]:
display( dfq.head() )

,bib,id,name,q_rank,q_speed,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points,r_points,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,points,qr_diff,q_split1_rank,q_split1_vs_best,q_split1_vs_winner,q_split2_rank,q_split2_vs_best,q_split2_vs_winner,q_split2_sector,q_split2_sector_rank,q_split2_sector_vs_best,q_split2_sector_vs_winner,q_split3_rank,q_split3_vs_best,q_split3_vs_winner,q_split3_sector,q_split3_sector_rank,q_split3_sector_vs_best,q_split3_sector_vs_winner,q_split4_rank,q_split4_vs_best,q_split4_vs_winner,q_split4_sector,q_split4_sector_rank,q_split4_sector_vs_best,q_split4_sector_vs_winner,q_split5_rank,q_split5_vs_best,q_split5_vs_winner,q_split5_sector,q_split5_sector_rank,q_split5_sector_vs_best,q_split5_sector_vs_winner,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner
0,2,2002,ATHERTON Rachel,16,49.86,74.580,116.942,160.266,212.906,249.889,2,40,Finished,10003434487,NaN,200.0,1.0,36.98,51.093,92.842,135.011,186.012,221.141,10.0,8.0,Finished,200.0,-28.748,37.0,22.124,0.000,31.0,19.111,0.000,42.362,1.0,0.000,0.000,30.0,20.540,0.000,43.324,3.0,1.429,0.000,18.0,21.165,0.000,52.640,3.0,1.043,0.000,16.0,22.661,0.000,36.983,5.0,1.496,0.000,1.0,0.000,0.000,1.0,0.000,0.000,41.749,2.0,0.492,0.000,1.0,0.000,0.000,42.169,1.0,0.000,0.000,2.0,0.503,0.000,51.001,2.0,1.004,0.000,1.0,0.000,0.000,35.129,2.0,0.715,0.000
1,1,2001,NICOLE Myriam,5,50.49,52.456,103.344,146.517,202.422,238.608,1,41,Finished,10004535237,20.0,160.0,2.0,36.18,51.993,93.250,135.512,185.509,221.779,13.0,5.0,Finished,180.0,-16.829,1.0,0.000,-22.124,8.0,5.513,-13.598,50.888,32.0,8.526,8.526,5.0,6.791,-13.749,43.173,2.0,1.278,-0.151,7.0,10.681,-10.484,55.905,14.0,4.308,3.265,5.0,11.380,-11.281,36.186,3.0,0.699,-0.797,2.0,0.900,0.900,2.0,0.408,0.408,41.257,1.0,0.000,-0.492,2.0,0.501,0.501,42.262,2.0,0.093,0.093,1.0,0.000,-0.503,49.997,1.0,0.000,-1.004,2.0,0.638,0.638,36.270,4.0,1.856,1.141
2,6,2006,HANNAH Tracey,2,50.98,55.436,99.536,143.504,195.101,230.608,5,37,Finished,10003732258,40.0,140.0,3.0,35.50,52.512,94.542,137.550,190.704,225.118,16.0,2.0,Finished,180.0,-5.490,3.0,2.980,-19.144,2.0,1.705,-17.406,44.100,5.0,1.738,1.738,2.0,3.778,-16.762,43.968,5.0,2.073,0.644,2.0,3.360,-17.805,51.597,1.0,0.000,-1.043,2.0,3.380,-19.281,35.507,2.0,0.020,-1.476,3.0,1.419,1.419,3.0,1.700,1.700,42.030,3.0,0.773,0.281,3.0,2.539,2.539,43.008,4.0,0.839,0.839,3.0,5.195,4.692,53.154,6.0,3.157,2.153,3.0,3.977,3.977,34.414,1.0,0.000,-0.715
3,9,2009,HRASTNIK Monika,4,49.48,56.999,100.942,144.646,198.395,235.390,8,34,Finished,10017602955,25.0,125.0,4.0,36.99,55.303,99.647,142.596,194.214,229.597,14.0,4.0,Finished,150.0,-5.793,5.0,4.543,-17.581,4.0,3.111,-16.000,43.943,2.0,1.581,1.581,4.0,4.920,-15.620,43.704,4.0,1.809,0.380,4.0,6.654,-14.511,53.749,5.0,2.152,1.109,4.0,8.162,-14.499,36.995,6.0,1.508,0.012,6.0,4.210,4.210,5.0,6.805,6.805,44.344,7.0,3.087,2.595,4.0,7.585,7.585,42.949,3.0,0.780,0.780,4.0,8.705,8.202,51.618,3.0,1.621,0.617,4.0,8.456,8.456,35.383,3.0,0.969,0.254
4,7,2007,SIEGENTHALER Emilie,3,49.48,55.776,99.845,144.556,197.387,235.296,6,36,Finished,10004167243,30.0,110.0,5.0,37.90,54.994,98.833,143.450,195.311,232.780,15.0,3.0,Finished,140.0,-2.516,4.0,3.320,-18.804,3.0,2.014,-17.097,44.069,4.0,1.707,1.707,3.0,4.830,-15.710,44.711,9.0,2.816,1.387,3.0,5.646,-15.519,52.831,4.0,1.234,0.191,3.0,8.068,-14.593,37.909,10.0,2.422,0.926,5.0,3.901,3.901,4.0,5.991,5.991,43.839,4.0,2.582,2.090,5.0,8.439,8.439,44.617,9.0,2.448,2.448,5.0,9.802,9.2

Now we have speed info either way, expand data set

In [98]:
dfq['speed_ms'] = dfq['speed'] * (1000/60/60)
dfq['speed_ms_vs_best'] = dfq['speed_ms'].max() - dfq.speed_ms
dfq['speed_rank'] = dfq.speed.rank(method='dense', ascending=False)
dfq['q_speed_rank'] = dfq['q_speed'].rank(method='dense', ascending=False)

# Data Export

All that's left is to save our data to CSV files so we can quickly import it again for analysis and visualization without making constant requests to the online servers. This not only reduces load on the services providing the data, but also allows us to work on our analysis "offline", moreover giving us a local copy in case the results are ever taken down. It's also much quicker to load data this way than constantly hitting online servers.

In [99]:
dfq.id = dfq.id.astype(str)
dfm = dfq.merge( df2, left_on='id', right_index=True, how='inner' )

In [100]:
df.to_csv( file_prefix + '.results.csv' )
dq.to_csv( file_prefix + '.quali.csv' )
df2.to_csv( file_prefix + '.racers.csv' )
dfm.to_csv( file_prefix + '.merged.csv' )

--- 

## Credits

### Author: Dominic Wrapson


> **@domwrap**
<br>
<img src="https://png.icons8.com/material/24/000000/github-2.png">
<img src="https://png.icons8.com/material/24/000000/stackoverflow.png">
<img src="https://png.icons8.com/material/24/000000/linkedin.png">
<img src="https://png.icons8.com/material/24/000000/windows8.png">
<img src="https://png.icons8.com/ios-glyphs/24/000000/instagram-new.png">
<img src="https://png.icons8.com/material/24/000000/twitter.png">
<a href="https://medium.com/@domwrap"><img src="https://png.icons8.com/material/24/000000/medium-logo.png"></a>
>
> <img src="https://png.icons8.com/material/24/000000/home.png"> http://domwrap.me
>
><img src="https://png.icons8.com/material/24/000000/cycling-mountain-bike.png"> [Hwulex](https://www.pinkbike.com/u/Hwulex/)


---

#### Special Thanks

Mark Shilton for the inspiration
- http://lookatthestats.blogspot.ca
- https://plus.google.com/+MarkShilton
- https://dirtmountainbike.com/author/mrgeekstats


<a href="https://icons8.com">Icon pack by Icons8</a>